# Preparación del Dataset
---

- Se obtienen y preprocesan textos directos del dataset para luego ser persistidos.

In [1]:
import pandas as pd
import os
from document import Document
import util.log as log
from multiprocessing import Process
from multiprocessing import Manager
from multiprocessing import Pool
import pandas as pd
log.init_logger()

In [2]:
corpora = []
errors = []

def process_file(file, corpus):
    try:
        path = '{}{}'.format(directory, file)
        document = Document(path)
        splitted_file = file.split(' $ ') 
        topic = splitted_file[0]
        title = splitted_file[1]
        corpus.append([path, title, document.string, document.word_count(), document.type_count(), document.sentences,
                document.lemmatized_string, document.stemmed_string, document.simple_preprocessed_string, topic, document.named_entities])
    except InvalidDocument:
        errors.append(file)

directory = 'labeled_dataset/'

files = os.listdir(directory)

ps = []

dataframe = None
shared_corpora = None
with Manager() as manager:

    shared_corpora = manager.list()

    for file in files[:2]:
        p = Process(target = process_file, args=(file,shared_corpora))
        p.start()
        ps.append(p)

    for p in ps:
        p.join()

    corpora = list(shared_corpora)


print('{} documents were processed. {} documents errored.'.format(len(corpora), len(errors)))

2 documents were processed. 0 documents errored.


In [ ]:
dataframe = pd.DataFrame(data = corpora, columns=['original_path', 'document_title', 'string', 'word_count', 'type_count', 'sentences','lemmatized_text', 'stemmed_text', 'simple_preprocessed', 'topic', 'named_entities'])